In [8]:
# imports
import pandas as pd
import torch
from chess import pgn
import io
import chess
from tqdm.notebook import tqdm

In [9]:
path = '/Users/anu/Documents/GM_games_dataset.csv'

In [10]:
df = pd.read_csv(path, nrows=100, usecols=['pgn'])


In [ ]:
# d = [king, queen, rook, bishop, knight, pawn, black, white, can_move]
def pgn_to_boardrep(game_pgn):
    game = pgn.read_game(game_pgn)
    moves = game.mainline_moves()
    board = chess.Board()
    pieces = chess.PIECE_TYPES
    states_arr = []
    moves_arr = []
    
    for move in moves:
        squares_arr = []
        for row in range(8):
            row_arr = []
            for col in range(8):
                square = (8 * row) + col
                col_arr = ([int(piece == board.piece_type_at(square)) for piece in pieces] + [int(board.turn == board.color_at(square))])
                row_arr.append(col_arr)
            squares_arr.append(row_arr)
        
        moves_arr.append([move.from_square, move.to_square])
        
        
        board.push(move)
        
        states_arr.append(squares_arr)

        
    return (torch.Tensor(states_arr, device='cpu'), torch.Tensor(moves_arr, device='cpu'))

In [12]:
features = torch.Tensor()
labels = torch.Tensor()
games = tqdm(df['pgn'].items(), total=len(df))
for game_pgn in games:
    features = torch.cat((features, pgn_to_boardrep(io.StringIO(game_pgn[1]))[0]))
    labels = torch.cat((labels, pgn_to_boardrep(io.StringIO(game_pgn[1]))[1]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
torch.save((features, labels), 'data/sample_dataset.pt')

In [14]:
features.shape

torch.Size([7481, 8, 8, 9])